<a href="https://colab.research.google.com/github/poorvitiwary/Computer_vision_Age/blob/main/training_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
print(tf.__version__)

TensorFlow is already loaded. Please restart the runtime to change versions.
2.4.1


In [27]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from sklearn.preprocessing import OneHotEncoder,LabelEncoder # Label encoding, 1-hot encoding, multi-encoding
# LABEL binarizer is a 1-hot encoded MATRIX 
import cv2
import numpy as np
import random
import imutils
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.layers import  GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [28]:
!ls data

'middle age'  'new born'   old	'young adult'


In [29]:
HP_dataset = 'data'
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'
HP_test_dataset = 'test'
HP_epoch = 100
HP_init_lr = 1e-3 # learning_rate = 0.001
HP_batch_size = 32
HP_image_dim = (96,96,3)

In [30]:
data = []
labels = [] 
# read all images
all_images = sorted(list(paths.list_images(HP_dataset)))
all_images[:5]

['data/middle age/middleage_0.png',
 'data/middle age/middleage_1.png',
 'data/middle age/middleage_10.png',
 'data/middle age/middleage_100.png',
 'data/middle age/middleage_101.png']

In [31]:
random.seed(42)
random.shuffle(all_images)
all_images[:5]


['data/old/oldage_328.png',
 'data/middle age/middleage_146.png',
 'data/new born/newborn_190.png',
 'data/young adult/youngadult_71.png',
 'data/old/oldage_255.png']

In [32]:
import os
for impath in all_images:
  img = cv2.imread(impath)
  resized = cv2.resize(img, (HP_image_dim[0],HP_image_dim[1]) )
  imageData = img_to_array(resized)
  data.append(imageData)
  # extract label from filename (2nd last element) / \\ 
  label = impath.split(os.path.sep)[-2]
  labels.append(label)

In [33]:
data = np.array(data, dtype='float' )/ 255.0
labels = np.array(labels)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0]])

In [34]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(data, labels, test_size=0.25, random_state=42)


In [35]:
#Augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.2, 
                         height_shift_range=0.2, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True,fill_mode='nearest')

In [36]:
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#N= 5
HP_block1_conv_dim = 32
HP_block2_conv_dim = 64
HP_block3_conv_dim = 128
HP_block4_conv_dim = 256
HP_block5_dense_dim = 1024
HP_small_pattern = (3,3) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (2,2) # 4 times compressed images from previous MP layers!!!
HP_dropout_little =0.25
HP_dropout_big = 0.50
# HP_epochs, batch_size-> are now problems of the developer USING this model. 

HP_img_dims = (96,96,3)


In [37]:
class RacoonVGG:
  @staticmethod
  def build(height, width, depth, classes):
    # assume that we are on TF, but if something else is detected, switch the dimension
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    model = Sequential()
    
    # BLOCK1
    model.add(Conv2D(HP_block1_conv_dim,HP_small_pattern, padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_small_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 2
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))
    
    # COMPLEX BLOCK 3
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 4
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())
    model.add(Dense(HP_block5_dense_dim))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(HP_dropout_big))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    return model


    # TF-> (h,w,d)
    # Theano-> (d,h,w)
    # GENERIC PROGRAM-> support BOTH the methods!!! 
    # if using KERAS-> because KERAS is just an API that is front facing an unknown
    # backend!

In [38]:
from tensorflow.keras.optimizers import Adam
model = RacoonVGG.build(height=HP_img_dims[0], width=HP_img_dims[1], depth=HP_img_dims[2],classes = len(lb.classes_))
opt = Adam(lr=HP_init_lr, decay = HP_init_lr/ HP_epoch)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [39]:
len(trainx)

728

In [40]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

In [42]:
history = model.fit(trainx, trainy, epochs=10, 
                    validation_data=(testx, testy),
                    callbacks = [early_stop])

Epoch 1/10
23/23 [==============================] - 28s 1s/step - loss: 2.6142 - accuracy: 0.2873 - val_loss: 1.3960 - val_accuracy: 0.3539
Epoch 2/10
23/23 [==============================] - 25s 1s/step - loss: 1.7938 - accuracy: 0.4713 - val_loss: 2.5437 - val_accuracy: 0.3539
Epoch 3/10
23/23 [==============================] - 25s 1s/step - loss: 1.4808 - accuracy: 0.4831 - val_loss: 3.4157 - val_accuracy: 0.3539
Epoch 4/10
23/23 [==============================] - 25s 1s/step - loss: 1.3415 - accuracy: 0.5253 - val_loss: 2.6195 - val_accuracy: 0.3539
Epoch 00004: early stopping
